<a href="https://colab.research.google.com/github/Nadezda-Sut/EEG/blob/main/MNE_Connectivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [262]:
!pip install mne==0.15.0



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mne: filename=mne-0.15-py3-none-any.whl size=6003955 sha256=63ead55eb93a5cb086be95a6bc0038f3904aa4686cfa44f9f9a53393b3746e01
  Stored in directory: /root/.cache/pip/wheels/61/0c/e3/1c5fe37b01821a55175908e6de4f40397c170f28d3938d002a
Successfully built mne
  Attempting uninstall: mne
    Found existing installation: mne 1.4.0
    Uninstalling mne-1.4.0:
      Successfully uninstalled mne-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autoreject 0.4.1 requires mne[hdf5]>=1.0, but you have mne 0.15 which is incompatible.


In [263]:
!pip install autoreject

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached mne-1.4.0-py3-none-any.whl (7.7 MB)
  Attempting uninstall: mne
    Found existing installation: mne 0.15
    Uninstalling mne-0.15:
      Successfully uninstalled mne-0.15


In [273]:

!pip install -U mne-connectivity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [279]:
import matplotlib
import pathlib
import mne
import pandas as pd
import numpy as np 
from autoreject  import AutoReject
ar = AutoReject()
from mne_connectivity import spectral_connectivity_time



In [ ]:
#main_folder = pathlib.Path().cwd()

#input_folder = pathlib.Path('D:/eeg_connectivity') 
#output_folder = pathlib.Path('D:/eeg_connectivity/output')



#files = [file for file in input_folder.iterdir() if file.suffix == '.fif']

#files = 'D:/eeg_connectivity/VatEvgDoICA.set'


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
     

# path to file in google drive

file_path = "/content/gdrive/My Drive/Colab Notebooks/VatEvgDoICA.set"
     

Mounted at /content/gdrive


In [4]:
input_file = '/content/gdrive/My Drive/Colab Notebooks/VatEvgDoICA.set'
output_file = '/content/gdrive/My Drive/Colab Notebooks/VatEvgDoICA.fif'

# Read the EEG data from the input file
raw = mne.io.read_raw_eeglab(input_file, preload=True)

# Save the EEG data to the output file in FIF format
raw.save(output_file, overwrite=True)

Reading /content/gdrive/My Drive/Colab Notebooks/VatEvgDoICA.fdt
Reading 0 ... 61924  =      0.000 ...   241.891 secs...
Overwriting existing file.
Writing /content/gdrive/My Drive/Colab Notebooks/VatEvgDoICA.fif


<ipython-input-4-664dc7b7bada>:5: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(input_file, preload=True)
<ipython-input-4-664dc7b7bada>:5: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(input_file, preload=True)
<ipython-input-4-664dc7b7bada>:8: RuntimeWarning: This filename (/content/gdrive/My Drive/Colab Notebooks/VatEvgDoICA.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(output_file, overwrite=True)


Closing /content/gdrive/My Drive/Colab Notebooks/VatEvgDoICA.fif
[done]


In [11]:
file = '/content/gdrive/My Drive/Colab Notebooks/VatEvgDoICA.fif'

# Convert .ced to .csv 

In [5]:
import csv
import re

def convert_ced_to_csv(ced_file, csv_file):
    # Open the .ced file for reading
    with open(ced_file, 'r') as ced:
        # Create a CSV writer object
        csv_writer = csv.writer(csv_file)
        
        # Write the header row
        csv_writer.writerow(["Electrode Label", "X", "Y", "Z"])
        
        # Read the .ced file line by line
        for line in ced:
            # Parse the electrode label and coordinates from each line
            match = re.match(r'(\w+)\s+([\d.-]+)\s+([\d.-]+)\s+([\d.-]+)', line)
            if match:
                electrode_label = match.group(1)
                x_coord = match.group(2)
                y_coord = match.group(3)
                z_coord = match.group(4)
                
                # Write the electrode information as a row in the CSV file
                csv_writer.writerow([electrode_label, x_coord, y_coord, z_coord])

# Example usage: Convert example.ced to example.csv
ced_file = '/content/gdrive/My Drive/Colab Notebooks/Channels.ced'
csv_file = '/content/gdrive/My Drive/Colab Notebooks/Channels.csv'

with open(csv_file, 'w', newline='') as output_csv:
    convert_ced_to_csv(ced_file, output_csv)

In [6]:
montage = mne.channels.read_custom_montage("/content/gdrive/My Drive/Colab Notebooks/Channels.csv")

In [8]:
epochs_duration = 3.
epochs_overlap = 0.5

con_methods = ['wpli']
#, 'imcoh', 'ciplv', 'ppc', 'coh' 
con_freqs = ['4-30','4-8','8-13','13-20', '20-30', '8-10', '10-13']

In [96]:
freqs_min = [int(x.split('-')[0]) for x in con_freqs]
freqs_max = [int(x.split('-')[1]) for x in con_freqs]

def epoching(raw, duration, overlap):
  tmin = 0
  tmax = duration
  duration = duration - overlap
  events=mne.make_fixed_length_events(raw, id=1,duration=duration) 
  picks = mne.pick_types( raw.info , meg=False, eeg=True, eog=False) 
  epochs = mne.Epochs(raw, events=events, tmin=tmin, baseline = (0, 0), 
                      tmax=tmax, picks=picks, reject_by_annotation=True)
  epochs.drop_bad() 
  return epochs


In [268]:
def calculate_conn(ts, con_methods, sfreq, freqs_min, freqs_max, node_names, person, output_folder): 
  for con_method in con_methods: 
    con, freqs, times, n_epochs, n_tapers = mne.connectivity.spectral_connectivity(ts, method=con_methods, mode='multitaper', sfreq=sfreq, 
                                                                                   fmin=freqs_min, fmax=freqs_max,
                                                                         faverage=True, mt_adaptive=False, n_jobs=1)
    sen_or_sour = 'sensors' if 'Fz' in node_names else 'sources'

    for shp in range(0, con.shape[2]): 
      output_filename = f'{output_folder}/{person}_{epochs_duration}-sec_{sen_or_sour}_{con_method}_{con_freqs[shp]}.csv' 
      savefile = pd.DataFrame(np.array(con[:, :, shp])) 
      savefile.columns = savefile.index = node_names

      savefile.to_csv(output_filename)

In [173]:
###############







In [33]:
raw = mne.io.read_raw_fif(file, preload=True)

Opening raw data file /content/gdrive/My Drive/Colab Notebooks/VatEvgDoICA.fif...
    Range : 0 ... 61924 =      0.000 ...   241.891 secs
Ready.
Reading 0 ... 61924  =      0.000 ...   241.891 secs...


<ipython-input-33-8f81574964f6>:1: RuntimeWarning: This filename (/content/gdrive/My Drive/Colab Notebooks/VatEvgDoICA.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(file, preload=True)


In [34]:
epochs = epoching(raw,4.,0.5)


Not setting metadata
69 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 69 events and 1025 original time points ...
1 bad epochs dropped


In [91]:
output_folder = pathlib.Path('/content/gdrive/My Drive/Colab Notebooks')

In [303]:
person = 'VatEvgDo_matrix'
freqs = raw.info['sfreq'] / 2

NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNn

In [307]:
########################################################################3

def calculate_conn(ts, con_methods, freqs_min, freqs_max, node_names, person, output_folder, freqs):
    for con_method in con_methods:
        con, freqs_out, times, n_epochs, n_tapers = spectral_connectivity_time(
            ts, freqs, method=con_method, mode='multitaper', sfreq=sfreq,
            fmin=freqs_min, fmax=freqs_max,
            faverage=True, n_jobs=1
        )
        sen_or_sour = 'sensors' if 'Fz' in node_names else 'sources'

        for shp in range(con.shape[2]):
            output_filename = f'{output_folder}/{person}_epochs-duration_sec_{sen_or_sour}_{con_method}_{freqs_out[shp]}.csv'
            savefile = pd.DataFrame(np.array(con[:, :, shp]), columns=node_names, index=node_names)

            savefile.to_csv(output_filename)


NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

In [310]:
con_methods = ['wpli']
con_freqs = ['4-30', '4-8', '8-13', '13-20', '20-30', '8-10', '10-13']
freqs_min = [int(x.split('-')[0]) for x in con_freqs]
freqs_max = [int(x.split('-')[1]) for x in con_freqs]
sen_or_sour = 'sensors'

def calculate_conn(data, con_methods, sfreq, freqs_min, freqs_max, node_names, person, output_folder):

    for con_method in con_methods:
        con = spectral_connectivity_epochs(data, method = con_methods,
                                                                                       mode = 'multitaper',
                                                                                       sfreq=sfreq, fmin=freqs_min,
                                                                                       fmax = freqs_max, faverage = True,
                                                                                       mt_adaptive = True, n_jobs = -1)
        con = con.get_data(output='dense')
    
    for shp in range(0, con.shape[2]):
        output_filename = f'{output_folder}/{person}_{sen_or_sour}_{con_method}.csv'
        savefile = pd.DataFrame(np.array(con[:,:,shp]))
        savefile.columns = savefile.index = node_names

        savefile.to_csv(output_filename)

epochs = epoching(raw, 6., 0.5) #type - numpy array
person = 'man'
calculate_conn(epochs, con_methods, epochs.info['sfreq'], freqs_min, freqs_max, epochs.info['ch_names'], person, output_folder)

Not setting metadata
43 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 43 events and 1537 original time points ...
0 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 1596 connections
    using t=0.000s..6.000s for estimation (1537 points)
    computing connectivity for the bands:
     band 1: 4.2Hz..30.0Hz (156 points)
     band 2: 4.2Hz..8.0Hz (24 points)
     band 3: 8.2Hz..13.0Hz (30 points)
     band 4: 13.2Hz..20.0Hz (42 points)
     band 5: 20.2Hz..30.0Hz (60 points)
     band 6: 8.2Hz..10.0Hz (12 points)
     band 7: 10.2Hz..13.0Hz (18 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: WPLI
    computing connectivity for epochs 1..1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


    computing connectivity for epochs 2..2


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


    computing connectivity for epochs 3..3


...

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


    computing connectivity for epochs 42..42
    computing connectivity for epochs 43..43


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


    assembling connectivity matrix
[Connectivity computation done]


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished


In [308]:
calculate_conn(epochs, con_methods, freqs_min, freqs_max, epochs.info ['ch_names'], person, output_folder, freqs)


Using data from preloaded Raw for 68 events and 1025 original time points ...
Replacing existing metadata with 3 columns
Using data from preloaded Raw for 68 events and 1025 original time points ...


KeyError: ignored